# Обработка резюме HH RU

In [ ]:
import pandas as pd
import json

In [ ]:
from parse_hh_data import download, parse
ids = download.resume_ids(area_id = 113, specialization_id = 4, search_period = 365, num_pages = 2)

var = []
for i in range(0, len(ids)):
    resume = download.resume(ids[i])
    resume = parse.resume(resume)
    var.append(resume)

import json
with open("data_file.json", "w", encoding='utf-8') as write_file:
    json.dump(var, write_file, ensure_ascii=False)
write_file.close()

In [2]:
with open('drive/MyDrive/LD final/data_file_it.json') as f:
  data = json.load(f)

In [12]:
data[0]

{'area': 'Москва',
 'birth_date': '23\xa0августа\xa01994',
 'education': [{'name': 'Московский государственный университет им. М.В. Ломоносова, Москва',
   'organization': 'Физический, Нелинейная оптика и лазерная физика',
   'year': 2018},
  {'name': 'Московский государственный университет им. М.В. Ломоносова, Москва',
   'organization': 'Физический, Физика',
   'year': 2016}],
 'education_level': 'Высшее образование (Магистр)',
 'experience': [{'description': 'Обязанности: Разработка программ и библиотек, обеспечивающих работу видео-стриминговых и транскодирующих приложений, дальнейшее их тестирование и отладка. Осуществление сопровождения внедренных программ и программных средств\n\nОсновные инструменты: C/C++, Python, Bash, FFmpeg, Git\n\nДостижения: Реализовал и внедрил в действующее ПО технологию адаптивной потоковой передачи данных (DASH)',
   'end': '01-09-2020',
   'position': 'Программист C/С++',
   'start': '01-03-2018'}],
 'gender': 'Мужчина',
 'language': [{'level': 'Родно

In [57]:
gender = []
area = []
spec_name = []
title = []
salary = []
edu = []
exp = []
skill_set = []
skills = []

for resume in data:
  gender.append(resume['gender'])
  area.append(resume['area'])
  title.append(resume['title'])
  salary.append(resume['salary']['amount'])
  edu.append(resume['education_level'])
  skill_set.append(resume['skill_set'])
  skills.append(resume['skills'])

  exp_temp = []
  for i in range(len(resume['experience'])):
    exp_temp.append(resume['experience'][i]['position'])
  exp.append(exp_temp)

  spec_temp = []
  for i in range(len(resume['specialization'])):
    spec_temp.append(resume['specialization'][i]['name'])
  spec_name.append(spec_temp)

In [58]:
import random as rd

prior = []
for i in range(len(data)):
  prior.append(rd.randint(0, 1))

In [59]:
df = pd.DataFrame([gender, area, title, salary, edu, skill_set, skills, exp, spec_name, prior]).transpose()

In [60]:
df.head()

,0,1,2,3,4,5,6,7,8,9
0,Мужчина,Москва,Специалист по машинному обучению,120000,Высшее образование (Магистр),"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++],"[Программирование, Разработка, Аналитик]",1
1,Мужчина,Москва,Разработчик React-Native JavaScript (Redux),None,Высшее образование,"[HTML, CSS, React-Native, JS6, Git, FireBase, ...","Из резюме удален весь нерелевантный опыт, чтоб...","[Разработчик React Native, Разработчик React N...","[Программирование, Разработка]",1
2,Мужчина,Санкт-Петербург,ИТ-специалист,None,Высшее образование,"[HTML, CSS, PHP, Сборка ПК, Настройка ПК, MySQ...",Окончил университет СПБГУТ в Санкт-Петербурге ...,"[Специалист технической поддержки, ИТ Специали...","[Банковское ПО, Программирование, Разработка, ...",0
3,Мужчина,Санкт-Петербург,Программист C#,None,Среднее образование,"[C#, Unity, Godot, SQL, MySQL, Git, .NET Frame...",Примеры моего кода и прочего\nhttps://github.c...,"[Оператор call-центра, Диспетчер технической п...","[Игровое ПО, Программирование, Разработка]",1
4,Женщина,Санкт-Петербург,Frontend-разработчик (JavaScript) junior,30000,Высшее образование,"[HTML5, CSS3, JavaScript, jQuery, CSS, Git, HT...",- Дипломы колледжа и бакалавра с отличием (кра...,"[Junior разработчик веб-интерфейсов, Оператор...","[Программирование, Разработка, Web инженер, We...",1


In [61]:
#gender, area, title, salary, edu, skill_set, skills, exp, spec_name, prior
df.rename(columns={0:'gender', 1:'area', 2:'title', 3:'salary',
                   4:'education', 5:'skill_set', 6:'skills', 7:'exp', 8:'spec_name', 9:'prior'}, inplace=True)

# Очистка данных (ДЗ)

In [62]:
df['education'].value_counts()

Высшее образование                    198
Неоконченное высшее образование        51
Высшее образование (Магистр)           36
Среднее специальное образование        34
Высшее образование (Бакалавр)          34
Образование                            16
Higher education                        5
Среднее образование                     4
Higher education (bachelor)             4
Высшее образование (Кандидат наук)      3
Education                               2
Secondary special education             2
Secondary education                     1
Name: education, dtype: int64

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gender     390 non-null    object
 1   area       390 non-null    object
 2   title      390 non-null    object
 3   salary     198 non-null    object
 4   education  390 non-null    object
 5   skill_set  390 non-null    object
 6   skills     390 non-null    object
 7   exp        390 non-null    object
 8   spec_name  390 non-null    object
 9   prior      390 non-null    object
dtypes: object(10)
memory usage: 30.6+ KB


In [64]:
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 198 entries, 0 to 389
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   gender     198 non-null    object
 1   area       198 non-null    object
 2   title      198 non-null    object
 3   salary     198 non-null    object
 4   education  198 non-null    object
 5   skill_set  198 non-null    object
 6   skills     198 non-null    object
 7   exp        198 non-null    object
 8   spec_name  198 non-null    object
 9   prior      198 non-null    object
dtypes: object(10)
memory usage: 17.0+ KB


In [65]:
df.loc[df.education=='Higher education', 'education'] = 'Высшее образование'

In [66]:
df.loc[df.gender=='Male', 'gender'] = 'Мужчина'
df.loc[df.gender=='Female', 'gender'] = 'Женщина'

In [67]:
df.head(1)

,gender,area,title,salary,education,skill_set,skills,exp,spec_name,prior
0,Мужчина,Москва,Специалист по машинному обучению,120000,Высшее образование (Магистр),"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++],"[Программирование, Разработка, Аналитик]",1


In [68]:
df['gender'].value_counts()

Мужчина    359
Женщина     31
Name: gender, dtype: int64

In [69]:
df = df.join(pd.get_dummies(df['gender']))

In [70]:
df.drop('gender', axis=1, inplace=True)

In [71]:
areas = list(df['area'].value_counts().index)

for i in range(len(areas)):
  df.loc[df['area']==areas[i], 'area'] = i

In [72]:
titles = list(df['title'].value_counts().index)

for i in range(len(titles)):
  df.loc[df['title']==titles[i], 'title'] = i

In [73]:
edus = list(df['education'].value_counts().index)

for i in range(len(edus)):
  df.loc[df['education']==edus[i], 'education'] = i

In [74]:
df.head(1)

,area,title,salary,education,skill_set,skills,exp,spec_name,prior,Женщина,Мужчина
0,0,227,120000,2,"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++],"[Программирование, Разработка, Аналитик]",1,0,1


In [78]:
df.drop('spec_name', axis=1, inplace=True)


In [79]:
df.head(1)

,area,title,salary,education,skill_set,skills,exp,prior,Женщина,Мужчина
0,0,227,120000,2,"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++],1,0,1


In [80]:
df['skill_set']

0      [Машинное обучение, Data Mining, Data Analysis...
1      [HTML, CSS, React-Native, JS6, Git, FireBase, ...
2      [HTML, CSS, PHP, Сборка ПК, Настройка ПК, MySQ...
3      [C#, Unity, Godot, SQL, MySQL, Git, .NET Frame...
4      [HTML5, CSS3, JavaScript, jQuery, CSS, Git, HT...
                             ...                        
385    [Пользователь ПК, MS Office, Работа в команде,...
386    [Unix, Администрирование серверов Linux, Unix ...
387    [Python, C/C++, Qt, MySQL, pytorch, OpenCV, Ma...
388    [1C: Бухгалтерия, MS SharePoint, Symantec Anti...
389                                                   []
Name: skill_set, Length: 390, dtype: object

In [85]:
import numpy as np

def skill_search(col, skills):
  """
  Функция для поиска ключевых слов в резюме
  """
  for skill in skills:
    df[skill] = 0
    skilled_ids = df[col].apply(lambda x: x if skill in str(x).lower() 
                                else np.nan).dropna().index
    df.loc[df.index.isin(skilled_ids), skill] = 1

In [86]:
skills_for_candidates = ['английский']
col = 'skills'
skill_search(col, skills_for_candidates)

In [91]:
exp = ['программист c', '++', 'react']
col = 'exp'
skill_search(col, exp)

In [92]:
df.head(1)

,area,title,salary,education,skill_set,skills,exp,prior,Женщина,Мужчина,английский,программист с,++,react,программист c
0,0,227,120000,2,"[Машинное обучение, Data Mining, Data Analysis...",Ответственное отношение к учебе\работе. \nОко...,[Программист C/С++],1,0,1,0,0,1,0,1


In [93]:
skills = ['машинное обучение', 'sql', 'data mining']
col = 'skill_set'
skill_search(col, skills)

In [96]:
df.drop(['skill_set', 'skills', 'exp'], axis=1, inplace=True)

In [97]:
df.drop('программист с', axis=1, inplace=True)

In [99]:
df.fillna(0, inplace=True)

In [100]:
x = df.drop('prior', axis=1)
y = df['prior']

In [101]:
from sklearn.model_selection import train_test_split

x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=.3, random_state=1)

In [102]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()
rfc.fit(x_tr, y_tr)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [105]:
proba = pd.DataFrame(rfc.predict_proba(x_ts))[1]


In [110]:
rfc.predict(x_ts)

array([0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0])

In [109]:
rfc.predict_proba(x_ts)

array([[0.81, 0.19],
       [0.64, 0.36],
       [0.3 , 0.7 ],
       [0.19, 0.81],
       [0.64, 0.36],
       [0.42, 0.58],
       [0.42, 0.58],
       [0.79, 0.21],
       [0.33, 0.67],
       [0.73, 0.27],
       [0.56, 0.44],
       [0.69, 0.31],
       [0.6 , 0.4 ],
       [0.43, 0.57],
       [0.25, 0.75],
       [0.78, 0.22],
       [0.8 , 0.2 ],
       [0.46, 0.54],
       [0.42, 0.58],
       [0.36, 0.64],
       [0.58, 0.42],
       [0.29, 0.71],
       [0.09, 0.91],
       [0.55, 0.45],
       [0.83, 0.17],
       [0.47, 0.53],
       [0.55, 0.45],
       [0.52, 0.48],
       [0.69, 0.31],
       [0.71, 0.29],
       [0.68, 0.32],
       [0.36, 0.64],
       [0.36, 0.64],
       [0.26, 0.74],
       [0.62, 0.38],
       [0.43, 0.57],
       [0.59, 0.41],
       [0.39, 0.61],
       [0.35, 0.65],
       [0.8 , 0.2 ],
       [0.41, 0.59],
       [0.71, 0.29],
       [0.23, 0.77],
       [0.36, 0.64],
       [0.88, 0.12],
       [0.66, 0.34],
       [0.19, 0.81],
       [0.73,

In [107]:
proba[:20]

0     0.19
1     0.36
2     0.70
3     0.81
4     0.36
5     0.58
6     0.58
7     0.21
8     0.67
9     0.27
10    0.44
11    0.31
12    0.40
13    0.57
14    0.75
15    0.22
16    0.20
17    0.54
18    0.58
19    0.64
Name: 1, dtype: float64

In [111]:
from sklearn.metrics import accuracy_score

accuracy_score(rfc.predict(x_ts), y_ts)

0.5042735042735043

# ДЗ

- Очистить данные
- Расставить prior в зависимости от технических скиллов и обучить модель на этой информации